In [1]:
import numpy as np
from scipy import constants
import tabulate
import pandas as pd
import matplotlib.pyplot as plt
from IonChainTools import calcPositions,lengthScale
from tweezer_functions import *

In [2]:
#Constants in SI units
eps0 = constants.epsilon_0 
m = 39.9626*constants.atomic_mass
c = constants.c
e = constants.e
hbar = constants.hbar
pi = np.pi

In [ ]:
N = 12
#tweezed ions is an adjustable parameter
tweezed_ions = [0,1,5,6,10,11]
omega_rf_r = 1.0 * pi * 1e6 *2
omega_a = omega_rf_r /8

NA = 0.6
wx = 2*pi*1e6
d = 5e-6
tweezer_wavelength = 532*1e9
P_opt = 15e-3
omega_tweezer = 2*pi*c/tweezer_wavelength
df = pd.read_csv('S_P_only.csv',sep = ",",encoding='UTF-8')

lambdares = df["wavelength (nm)"]
lambdares = [i*1e-9 for i in lambdares]
omegares = [2*pi*c/i for i in lambdares]
print(omegares)
lifetimes = df["A_ki (s^-1)"]
lifetimes = [i for i in lifetimes]
omega_res = np.array(omegares)
linewidths = np.array(lifetimes)


beam_waist = 0.41*tweezer_wavelength / NA
#i'm not sure what the line under here is supposed to be for, I can't remember
#omega_tweezer_r = omega_tweezer_r(potential(omega_tweezer,linewidths,omega_res,P_opt,beam_waists),beam_waists,m)



In [ ]:
def combined_frequencies(N,tweezed_ions,w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a):


    omeg_tweezer_r = np.zeros(N)
    omeg_tweezer_a = np.zeros(N)
    omeg_tweezer_r[tweezed_ions] = w_tweezer_r
    omeg_tweezer_a[tweezed_ions] = w_tweezer_a

    omeg_rf_r = w_rf_r * np.ones(N) 
    omeg_rf_a = w_rf_a * np.ones(N)

    omega_combined_rr = np.sqrt(omeg_rf_r**2 + omeg_tweezer_r**2)
    omega_combined_ra = np.sqrt(omeg_rf_r**2 + omeg_tweezer_a)
    omega_combined_ar = np.sqrt(omeg_rf_a**2 + omeg_tweezer_r**2)
    
    return np.array([omega_combined_rr,omega_combined_ra,omega_combined_ar])

def trapping_ratios(w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a):
    
    tweezer_r_to_rf_ratio = w_tweezer_r / w_rf_r
    tweezer_r_to_axial_ratio = w_tweezer_r / w_rf_a
    tweezer_a_to_rf_ratio = w_tweezer_a / w_rf_a
    
    return np.array([tweezer_r_to_rf_ratio,tweezer_r_to_axial_ratio])

In [ ]:
def potential(omega_tweezer,linewidths,omega_res,P_opt,beam_waist):
    '''
    Find the potential of the optical tweezers beam for
    the given set of parameters at r=0 and z=0 -- without RWA


    omega_tweezer = angular frequency of tweezer laser beam
    linewidths = linewidth of the given resonant transition of ion taken from NIST database in angular frequency units 
        (ex, S1/2 to P1/2 and S1/2 to P3/2 for 40Ca+), this is a list/array of some form with all relevent transitions
    P_opt = total optical power of tweezer laser beam
    omega_res = angular frequency of resonant transition, also based off NIST data
    beam_waists = beamwaist of the tweezer laser beam
    given its frequency and the NA of our system or from measurement
    '''
    p = []
    for i in range(len(linewidths)): 
        p.append( (-3*P_opt*(c**2)/((omega_res[i]**3)*(beam_waist**2))) * (linewidths[i]/((omega_res[i] - omega_tweezer)) +
                                          linewidths[i]/(omega_res[i] + omega_tweezer)) ) 
    pot = sum(p)
    return pot

In [ ]:
potential(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)

In [ ]:
def potentialRWA(omega_tweezer,linewidths,omega_res,P_opt,beam_waist):
    '''
    Find the potential of the optical tweezers beam for
    the given set of parameters at r=0 and z=0 -- with RWA


    omega_tweezer = angular frequency of tweezer laser beam
    linewidths = linewidth of the given resonant transition of ion taken from NIST database in angular frequency units 
        (ex, S1/2 to P1/2 and S1/2 to P3/2 for 40Ca+), this is a list/array of some form with all relevent transitions
    P_opt = total optical power of tweezer laser beam
    omega_res = angular frequency of resonant transition, also based off NIST data
    beam_waists = beamwaist of the tweezer laser beam
    given its frequency and the NA of our system or from measurement
    '''
    p = []
    for i in range(len(linewidths)): 
        p.append( (-3*P_opt*(c**2)/((omega_res[i]**3)*(beam_waist**2))) * (linewidths[i]/((omega_res[i] - omega_tweezer)))  ) 
    pot = sum(p)
    return pot

In [ ]:
potentialRWA(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)

In [ ]:
test = []
for i in range(len(linewidths)):
    test.append ((-3*P_opt*(c**2)/((omega_res[i]**3)*(beam_waist**2))) *
                                          (linewidths[i]/(omega_res[i] + omega_tweezer)) )
sum(test)    


In [ ]:
 potentialRWA(omega_tweezer,linewidths,omega_res,P_opt,beam_waist) + -1.6996758171451078e-62 ==  potential(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)

In [ ]:
def scattering(omega_tweezer,linewidths,omega_res,P_opt,beam_waist):
    '''
    Find the scattering of the optical tweezers beam (at r=0 and z=0) off of a given resonance
    for the given set of parameters -- without RWA
    omega_tweezer = angular frequency of tweezer laser beam
    linewidths = linewidth of the given resonant transition taken from NIST database in angular frequency units 
        (ex, S1/2 to P1/2 and S1/2 to P3/2 for 40Ca+)
    P_opt = total optical power of tweezer laser beam
    omega_res = angular frequency of resonant transition, also based off NIST data
    beam_waists = beamwaist of the tweezer laser beam
    given its frequency and the NA of our system or from measurement
    '''
    s = []
    for i in range(len(linewidths)):
        s.append(((3*(c**2)*P_opt)/(hbar *pi* (omega_res[i]**3)*(beam_waist**2))) *((omega_tweezer/omegares[i])**3)* (((linewidths[i]/(omega_res[i] - omega_tweezer))+
                                                                            (linewidths[i]/(omegares[i] + omega_tweezer)))**2) )
    scat = sum(s)
    return scat

In [ ]:
scattering(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)

In [ ]:
def scatteringRWA(omega_tweezer,linewidths,omega_res,P_opt,beam_waist):
    '''
    Find the scattering of the optical tweezers beam (at r=0 and z=0) off of a given resonance
    for the given set of parameters -- with RWA
    omega_tweezer = angular frequency of tweezer laser beam
    linewidths = linewidth of the given resonant transition taken from NIST database in angular frequency units 
        (ex, S1/2 to P1/2 and S1/2 to P3/2 for 40Ca+)
    P_opt = total optical power of tweezer laser beam
    omega_res = angular frequency of resonant transition, also based off NIST data
    beam_waists = beamwaist of the tweezer laser beam
    given its frequency and the NA of our system or from measurement
    '''
    s = []
    for i in range(len(linewidths)):
        s.append( (3*P_opt*(c**2)/(hbar *pi* (omega_res[i]**3)*(beam_waist**2))) *((omega_tweezer/omega_res[i])**3) *((linewidths[i]/(omega_res[i] - omega_tweezer))**(2)) 
        )
    scat = sum(s)
    return scat



In [ ]:
scatteringRWA(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)

In [ ]:

N = 5
tweezed_ions = [0,4]
w_tweezer_r = 2*pi*1e6
w_tweezer_a = 2*pi*0.5*1e6
w_rf_r = 2*pi*1e6
w_rf_a = 2*pi*0.5e6
"""def combined_freqs_to_mode_vectors(N,tweezed_ions,w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a):




    combined_freqs = combined_frequencies(N,tweezed_ions,w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a)


    omega_r_combined = combined_freqs[0]
    omega_a = omega_rf_a

    return mode_calc_r(m,omega_r_combined,omega_a)"""

In [ ]:
combined_freqs_to_mode_vectors(N,tweezed_ions,w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a)

In [3]:
N = 12
#tweezed ions is an adjustable parameter
tweezed_ions = [0,1,5,6,10,11]
w_rf_r = 1.0 * pi * 1e6 *2
w_rf_a = w_rf_r /8

NA = 0.6
wx = 2*pi*1e6
d = 5e-6
tweezer_wavelength = 532*1e9
P_opt = 15e-3
omega_tweezer = 2*pi*c/tweezer_wavelength
df = pd.read_csv('S_P_only.csv',sep = ",",encoding='UTF-8')

lambdares = df["wavelength (nm)"]
lambdares = [i*1e-9 for i in lambdares]
omegares = [2*pi*c/i for i in lambdares]
print(omegares)
lifetimes = df["A_ki (s^-1)"]
lifetimes = [i for i in lifetimes]
omega_res = np.array(omegares)
linewidths = np.array(lifetimes)


beam_waist = 0.41*tweezer_wavelength / NA

"""
def physical_params_to_mode_vectors(omega_tweezer,linewidths,omega_res,w_rf_a,w_rf_r,P_opt,beam_waist,m):
    U = potential(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)
    w_tweezer_r =  omega_tweezer_r(U,beam_waist,m)
    w_tweezer_a = omega_tweezer_a(U,beam_waist,tweezer_wavelength,m)
    return combined_freqs_to_mode_vectors(N,tweezed_ions,w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a)"""

[4788543317790194.0, 4746544743851730.0]


'\ndef physical_params_to_mode_vectors(omega_tweezer,linewidths,omega_res,w_rf_a,w_rf_r,P_opt,beam_waist,m):\n    U = potential(omega_tweezer,linewidths,omega_res,P_opt,beam_waist)\n    w_tweezer_r =  omega_tweezer_r(U,beam_waist,m)\n    w_tweezer_a = omega_tweezer_a(U,beam_waist,tweezer_wavelength,m)\n    return combined_freqs_to_mode_vectors(N,tweezed_ions,w_tweezer_r,w_tweezer_a,w_rf_r,w_rf_a)'

In [4]:
physical_params_to_mode_vectors(omega_tweezer,linewidths,omega_res,w_rf_a,w_rf_r,P_opt,beam_waist,m)

NameError: name 'tweezer_wavelength' is not defined